# Optimal power flow model in Julia

In [ ]:
import Pkg

# activate parent environment
Pkg.activate(normpath(joinpath(@__DIR__, ".")))
Pkg.resolve()
Pkg.instantiate()
Pkg.status()

using CSV
using DataFrames
using Dates
using CairoMakie
using JuMP
import JSON
import MathOptInterface as MOI

using Revise
using OptHP

  Activating project at `c:\Users\lange\OneDrive\Projects\2024\EEM25\Congestion-Management-based-on-Thermal-Comfort`


## Load data

In [ ]:
network = CSV.read("data/network.csv", DataFrame)
# network = CSV.read("data/TestNet(in).csv", DataFrame)

# correct StarNode and EndNode for 1 based indexing
network.StartNode = network.StartNode .+ 1
network.EndNode = network.EndNode .+ 1

first(network, 5)

In [ ]:
connections = CSV.read("data/user_connect.csv", DataFrame; delim=";")
# connections = CSV.read("data/user_connect_test.csv", DataFrame; delim=";")

# correct for 1 based indexing
connections.Node = connections.Node .+ 1

# # convert PV (str) to Float64
connections.PV = parse.(Float64, replace.(connections.PV, "," => "."))

first(connections, 5)

In [ ]:
loads_real = CSV.read("data/UserPower.csv", DataFrame)
loads_real.time = DateTime.(loads_real.time, "m/d/yyyy H:M p")

loads_reactive = CSV.read("data/UserReactivePower.csv", DataFrame)
loads_reactive.time = DateTime.(loads_reactive.time, "m/d/yyyy H:M p")

# Filter for rows where the date is February 1, 2024
date = Date(2024, 2, 1)
loads_real = loads_real[Date.(loads_real.time) .== date, 3:end]
loads_reactive = loads_reactive[Date.(loads_reactive.time) .== date, 3:end]
first(loads_real, 5)

## Build graph model

In [ ]:
# Step 1: Identify all unique nodes
nodes = 0:maximum(vcat(network.EndNode, network.StartNode))
node_indices = Dict(node => i for (i, node) in enumerate(nodes))  # Map nodes to indices

# Step 2: Initialize an adjacency matrix
n = length(nodes)
adjacency_matrix = zeros(Int, n, n)

# Step 3: Populate the matrix with connections
for (start_node, end_node) in zip(network.StartNode, network.EndNode)
    j = node_indices[start_node]
    i = node_indices[end_node]
    adjacency_matrix[i, j] = 1
end

# this gives:
# --- StartNode ---
#   0, 1, 2, 3, ...
# 0 1, 0, 0, 0, ... 
# 1 0, 0, 1, 0, ...
# 2 0, 0, 0, 1, ...
df_adj = DataFrame(adjacency_matrix, string.(nodes))

# save as CSV
CSV.write("data/adjacency_matrix.csv", df_adj, delim=",")

## Construct GEC

In [ ]:
model = GEC(network=network, 
            connections=connections, 
            loads_real=loads_real .* 1E-3, 
            loads_reactive=loads_reactive .* 1E-3,        
)

In [ ]:
solution_summary(model)

In [ ]:
function prepare_solution(model)
    # prepare JuMP solution for plotting
    sol = Dict{Symbol, Vector{Float64}}()
    for var in [:P, :Q, :P_pv, :P_hp, :P_hp_down, :P_pv_down]
        sol[var] = vec(sum(Matrix{Float64}(value.(model[var]; result=1)), dims=2)) .* 1E3
    end

    # some solutions have to be corrected
    for var in [:P, :Q]
        sol[var] = sol[var] .* -1
    end
    return sol
end

In [ ]:
sol = prepare_solution(model)

# plot using Makie
fig = Figure(; size = (1000, 600))
ax = Axis(fig[1, 1], xlabel = "Time [hours]", 
    ylabel = "Power [kW]", 
    title = "Power flow",
    xticks = (1:4:97, string.(0:1:24))
)

P_trafo = Matrix(value.(model[:P]; result=1))[:, 1] .* -1E3

# lines!(ax, sol[:P], color = :blue, label = "Transformer", linestyle = :so
scatterlines!(ax, P_trafo, color = :blue, label = "Transformer", linewidth = 2)
# scatterlines!(ax, sol[:Q], color = :green, label = "Reactive power", linewidth = 2)
scatterlines!(ax, sol[:P_hp], color = :red, label = "Heat pump", linewidth = 2)
scatterlines!(ax, sol[:P_pv], color = :orange, label = "PV", linewidth = 2)
scatterlines!(ax, sol[:P_hp_down], color = :purple, label = "Heat pump (down)", marker=:cross, linewidth = 2)
scatterlines!(ax, sol[:P_pv_down], color = :black, label = "PV (down)", marker=:x, linewidth = 2)
# fig[1, 2] = Legend(fig, ax)
axislegend(ax, merge = true, position = :lb)

# display
fig

In [ ]:
P_load_user = Vector{Float64}(value.(model[:P_load_user]; result=1)) .* 1E3
P_loss = Vector{Float64}(value.(model[:P_loss]; result=1)) .* 1E3
P_trafo = Matrix(value.(model[:P]; result=1))[:, 1] .* -1E3

# plot using Makie
fig_load = Figure(; size = (1000, 600))
axl = Axis(fig_load[1, 1], xlabel = "Time [hours]", 
    ylabel = "Power [kW]", 
    title = "Power flow",
    xticks = (1:4:97, string.(0:1:24))
)

scatterlines!(axl, P_trafo, color = :blue, label = "Transformer", linewidth = 2)
scatterlines!(axl, P_load_user, color = :purple, label = "P_load_user", linewidth = 2)
scatterlines!(axl, P_loss, color = :red, label = "P_loss", linewidth = 2)
axislegend(axl, merge = true, position = :lb)

# display
fig_load

In [ ]:
V = Matrix(value.(model[:V]; result=1))
V_ref = 0.23 # [kV]

I = Matrix(value.(model[:I]; result=1)) .* 1E3

# plot using Makie
fig_volt = Figure(; size = (1000, 600))
axv = Axis(fig_volt[1, 1], xlabel = "Time [hours]", 
    ylabel = "Voltage [V]", 
    title = "Voltage",
    xticks = (1:4:97, string.(0:1:24))
)

# scatterlines!(axv, V, color = :blue, label = "Voltage", linewidth = 2)
for i in 1:size(V, 2)
    scatterlines!(axv, sqrt.(V[:, i]), color = :blue, label = "Voltage", linewidth = 2)
end

# twin axis 
axv2 = Axis(fig_volt[1, 1], xlabel = "Time [hours]", 
    ylabel = "Current [A]", 
    title = "Current",
    xticks = (1:4:97, string.(0:1:24)),
    # move axis to the right
    yaxisposition = :right
)
for i in 1:size(I, 2)
    scatterlines!(axv2, I[:, i], color = :red, label = "Current", linewidth = 2)
end
axislegend(axv, merge = true, position = :lb)


# display
fig_volt

## Write model to LP file

In [20]:
# model = MOI.FileFormats.Model(format = MOI.FileFormats.FORMAT_LP)
# src = MOI.Utilities.Model{Float64}()
dest = MOI.FileFormats.Model(format = MOI.FileFormats.FORMAT_LP);
MOI.copy_to(dest, model)
MOI.write_to_file(dest, "model_jl_test.txt")

In [ ]:
solution = Dict(name(x) => value(x) for x in all_variables(model))
write("solution.json", JSON.json(solution))